In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

In [2]:
dfFlight = pd.read_csv('FlightDelayWithWeather.csv')

In [3]:
dfFlight.head()

,DEPARTURE_DELAY,Mon,Tue,Wed,Thu,Fri,Sat,AA,AS,B6,...,VX,Air_Temperature,Dew_Point_Temperature,Relative_Humidity,Wind_Direction,Wind_Speed,Precipitation_Per_Hour,Pressure_Altimeter,Sea_Level_Pressure,Visibility
0,0,0,0,0,1,0,0,0,0,0,...,0,53.06,32.00,44.48,280.0,7.0,0.0,30.09,1018.9,10.0
1,0,0,0,0,1,0,0,0,0,0,...,0,53.06,32.00,44.48,280.0,7.0,0.0,30.09,1018.9,10.0
2,0,0,0,0,1,0,0,1,0,0,...,0,53.06,32.00,44.48,280.0,7.0,0.0,30.09,1018.9,10.0
3,0,0,0,0,1,0,0,0,0,0,...,0,53.06,32.00,44.48,280.0,7.0,0.0,30.09,1018.9,10.0
4,1,0,0,0,1,0,0,0,0,0,...,0,44.96,35.96,70.52,150.0,4.0,0.0,30.08,1018.7,10.0


In [4]:
X = np.array(dfFlight.drop(['DEPARTURE_DELAY'], axis = 1).values)
Y = np.array(dfFlight['DEPARTURE_DELAY'].values)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(110452, 25)
(27614, 25)
(110452,)
(27614,)


In [5]:
def normalizer(data):
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(data)
    df_normalized = pd.DataFrame(np_scaled)
    return df_normalized

In [6]:
X_train = normalizer(X_train)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.667722,0.795082,0.463511,0.555556,0.40625,0.0,0.346667,0.345098,1.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.439873,0.729508,0.732769,0.000000,0.00000,0.0,0.560000,0.552941,1.000000
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.702532,0.319672,0.085380,0.750000,0.50000,0.0,0.373333,0.368627,1.000000
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.281646,0.684426,1.000000,0.000000,0.00000,0.0,0.440000,0.439216,0.493927
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.455696,0.819672,0.842476,0.833333,0.18750,0.0,0.346667,0.349020,1.000000


In [7]:
X_test = normalizer(X_test)
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.408360,0.180328,0.200215,0.972222,0.12500,0.0,0.733333,0.729412,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.482315,0.774590,0.734677,0.555556,0.21875,0.0,0.266667,0.266667,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.607717,0.729508,0.483186,0.722222,0.25000,0.0,0.360000,0.364706,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.408360,0.590164,0.603506,0.694444,0.43750,0.0,0.386667,0.380392,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.266881,0.659836,1.000000,0.416667,0.25000,0.0,0.320000,0.317647,1.0


In [8]:
def calc_score(clf,X, Y):#, _penalty, _solver, _multi_class):
    #clf=KNeighborsClassifier(n_neighbors=332,weights='distance',algorithm='kd_tree',p=1,metric='manhattan')#penalty=_penalty, solver = _solver, multi_class = _multi_class,random_state=9)
    score = np.mean(cross_val_score(clf, X, Y))
    return score

In [9]:
clf=KNeighborsClassifier(n_neighbors=332,weights='distance',algorithm='kd_tree')

In [10]:
score = calc_score(clf,X_train, Y_train)
score

0.6365027821800187

In [11]:
clf.fit(X_train,Y_train)
labels=clf.predict(X_test)
print(metrics.accuracy_score(Y_test,labels))

0.6477873542406026
